In [ ]:
from pathlib import Path

import pandas as pd
from misc import model_config

In [2]:
main_model_config = (
    model_config.query("main")
    .drop(columns="main")
    .rename(columns={k: f"model_{k}" for k in model_config.columns})
)

new_name = {
    "baseline": "Baseline",
    "32expert": "FewerExp",
    "top16": "ActMore",
    "top2": "ActFewer",
    "share1": "1ShrExp",
    "share2": "2ShrExp",
    "skip1": "DenseFst",
    "sparse2": "DenseHlf",
    "nolb": "NoLB",
    "overlb": "OverLB",
}

model_colors = {
    key: px.colors.qualitative.Dark24[i] for i, key in enumerate(main_model_config.index.values)
}

main_model_config

,model_name,model_abbr,model_type,model_num_params,model_num_layers,model_num_experts,model_top_k,model_attn
key,,,,,,,,
baseline,baseline,BL,causal,1.43,8,64,8,sdpa
32expert,baseline,FE,causal,1.43,8,32,4,sdpa
top16,baseline,AM,causal,1.43,8,64,16,sdpa
top2,baseline,AF,causal,1.43,8,64,2,sdpa
share1,share1,SH1,causal,1.43,8,63,7,sdpa
share2,share2,SH2,causal,1.43,8,62,6,sdpa
skip1,skip1,DF,causal,1.43,8,64,8,sdpa
sparse2,sparse2,DH,causal,1.43,8,64,8,sdpa
nolb,baseline,NB,causal,1.43,8,64,8,sdpa


In [3]:
def make_abbr(df):
    return (
        f"{df['model_abbr']}{'d' if df['is_decoder'] else 'e'}"
        if df["model_type"] == "seq2seq"
        else df["model_abbr"]
    )

In [ ]:
root_dir = Path("../output/tp_mpq")

df = pd.merge(
    pd.read_parquet(root_dir / "m.parquet"), main_model_config, left_on="model", right_index=True
)

df["model"] = df["model"].astype(model_config.index.dtype)
df

,model,dataset,cache_m,mode,prefill_thrp,decode_thrp,model_name,model_abbr,model_type,model_num_params,model_num_layers,model_num_experts,model_top_k,model_attn
0,baseline,c4,1.0,old_doc,669.586534,10.345799,baseline,BL,causal,1.43,8,64,8,sdpa
1,baseline,c4,1.0,new_gen,669.762733,10.473391,baseline,BL,causal,1.43,8,64,8,sdpa
2,baseline,c4,2.0,old_doc,746.376371,11.608819,baseline,BL,causal,1.43,8,64,8,sdpa
3,baseline,c4,2.0,new_gen,746.203748,11.705259,baseline,BL,causal,1.43,8,64,8,sdpa
4,baseline,c4,3.0,old_doc,803.713408,12.458667,baseline,BL,causal,1.43,8,64,8,sdpa
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1051,overlb,science,2.0,new_gen,747.422945,11.373545,baseline,OB,causal,1.43,8,64,8,sdpa
1052,overlb,science,3.0,old_doc,805.651832,12.072060,baseline,OB,causal,1.43,8,64,8,sdpa
1053,overlb,science,3.0,new_gen,803.538583,12.103101,baseline,OB,causal,1.43,8,64,8,sdpa
1054,overlb,science,inf,old_doc,1408.173687,18.671890,baseline,OB,causal,1.43,8,64,8,sdpa


In [ ]:
mdf = df.groupby(["model", "cache_m", "mode"], observed=True, as_index=False)[
    ["prefill_thrp", "decode_thrp"]
].mean()

mdf.pivot(index=["model", "mode"], columns="cache_m", values=["prefill_thrp", "decode_thrp"])

prefill_thrp                                        \
cache_m                   1.0         2.0          3.0          inf   
model    mode                                                         
baseline old_doc   671.211354  746.142357   806.701947  1440.988898   
         new_gen   671.105930  750.026480   805.326465  1442.081170   
32expert old_doc   687.171915  733.765020   785.751764  1445.474156   
         new_gen   687.158222  735.172242   786.745651  1446.816350   
top16    old_doc   726.672124  850.446746  1049.297554  1370.380943   
         new_gen   727.987611  851.126996  1050.449929  1371.243004   
top2     old_doc   687.464493  687.195766   707.438857  1524.063953   
         new_gen   690.125515  689.232838   707.698981  1523.566155   
share1   old_doc   674.368386  732.038827   812.139154  1444.953554   
         new_gen   672.579744  730.609584   813.440267  1446.292447   
share2   old_doc   687.200293  730.814858   813.004885  1410.196969   
         new_gen   685.065024  729.386741   811.455967  1410.665727   
skip1    old_doc   697.463600  745.015355   801.273074  1404.493624   
         new_gen   699.439137  748.287358   804.287507  1398.983069   
sparse2  old_doc   675.036810  736.054379   815.628419  1433.227215   
         new_gen   674.951998  733.990816   815.663027  1434.332329   
nolb     old_doc   691.206221  733.169418   809.907095  1445.865037   
         new_gen   689.752999  734.263462   809.734258  1447.561947   
overlb   old_doc   693.610061  744.817100   802.946345  1408.730270   
         new_gen   692.311156  743.685439   803.200889  1409.262910   

                 decode_thrp                                   
cache_m                  1.0        2.0        3.0        inf  
model    mode                                                  
baseline old_doc   10.570958  12.041458  13.023481  19.036706  
         new_gen   10.639580  12.135196  13.099395  19.029482  
32expert old_doc   10.795390  11.803518  12.719432  19.093189  
         new_gen   10.853180  11.872818  12.816151  19.086573  
top16    old_doc    6.395263   7.618822   8.985702  10.777566  
         new_gen    6.451738   7.689991   9.046690  10.769219  
top2     old_doc   27.467790  27.630046  28.598128  41.807579  
         new_gen   27.510340  27.657158  28.643450  41.795898  
share1   old_doc   10.647874  11.811415  13.060729  19.074727  
         new_gen   10.719114  11.873119  13.142425  19.069902  
share2   old_doc   10.848017  11.822392  13.094812  18.612788  
         new_gen   10.915339  11.871950  13.178700  18.608064  
skip1    old_doc   10.911991  12.016083  12.891599  18.516184  
         new_gen   10.991878  12.085808  12.960800  18.429879  
sparse2  old_doc   10.657725  11.844485  13.100840  18.946540  
         new_gen   10.719985  11.890266  13.206035  18.941017  
nolb     old_doc   10.958086  11.934119  13.089345  19.086657  
         new_gen   11.016366  12.018020  13.164880  19.095608  
overlb   old_doc   10.527826  11.368768  12.129843  18.652153  
         new_gen   10.576435  11.389267  12.185365  18.645716

In [16]:
mmdf = mdf.query("cache_m != inf").merge(
    mdf.query("cache_m == inf").drop(columns="cache_m"), on=["model", "mode"], suffixes=("", "_b")
)

mmdf["prefill_oh"] = 1 / mmdf["prefill_thrp"] - 1 / mmdf["prefill_thrp_b"]
mmdf["decode_oh"] = 1 / mmdf["decode_thrp"] - 1 / mmdf["decode_thrp_b"]
mmdf["prefill_ohr"] = mmdf["prefill_oh"] * mmdf["prefill_thrp_b"]
mmdf["decode_ohr"] = mmdf["decode_oh"] * mmdf["decode_thrp_b"]

mmdf.pivot(
    columns="cache_m",
    index=["model", "mode"],
    values=["prefill_oh", "decode_oh", "prefill_ohr", "decode_ohr"],
)

prefill_oh                     decode_oh                      \
cache_m                 1.0       2.0       3.0       1.0       2.0       3.0   
model    mode                                                                   
baseline old_doc   0.000796  0.000646  0.000546  0.042069  0.030516  0.024254   
         new_gen   0.000797  0.000640  0.000548  0.041439  0.029855  0.023789   
32expert old_doc   0.000763  0.000671  0.000581  0.040257  0.032346  0.026245   
         new_gen   0.000764  0.000669  0.000580  0.039746  0.031833  0.025634   
top16    old_doc   0.000646  0.000446  0.000223  0.063580  0.038469  0.018503   
         new_gen   0.000644  0.000446  0.000223  0.062140  0.037182  0.017680   
top2     old_doc   0.000798  0.000799  0.000757  0.012487  0.012273  0.011048   
         new_gen   0.000793  0.000795  0.000757  0.012424  0.012231  0.010986   
share1   old_doc   0.000791  0.000674  0.000539  0.041490  0.032238  0.024140   
         new_gen   0.000795  0.000677  0.000538  0.040853  0.031785  0.023651   
share2   old_doc   0.000746  0.000659  0.000521  0.038456  0.030859  0.022640   
         new_gen   0.000751  0.000662  0.000523  0.037874  0.030492  0.022140   
skip1    old_doc   0.000722  0.000630  0.000536  0.037635  0.029215  0.023563   
         new_gen   0.000715  0.000622  0.000529  0.036717  0.028482  0.022896   
sparse2  old_doc   0.000784  0.000661  0.000528  0.041049  0.031647  0.023551   
         new_gen   0.000784  0.000665  0.000529  0.040488  0.031307  0.022927   
nolb     old_doc   0.000755  0.000672  0.000543  0.038864  0.031401  0.024005   
         new_gen   0.000759  0.000671  0.000544  0.038406  0.030840  0.023592   
overlb   old_doc   0.000732  0.000633  0.000536  0.041373  0.034347  0.028828   
         new_gen   0.000735  0.000635  0.000535  0.040918  0.034170  0.028434   

                 prefill_ohr                     decode_ohr            \
cache_m                  1.0       2.0       3.0        1.0       2.0   
model    mode                                                           
baseline old_doc    1.146848  0.931252  0.786272   0.800850  0.580930   
         new_gen    1.148813  0.922707  0.790679   0.788556  0.568123   
32expert old_doc    1.103512  0.969941  0.839607   0.768643  0.617585   
         new_gen    1.105507  0.967996  0.838989   0.758616  0.607586   
top16    old_doc    0.885831  0.611366  0.305998   0.685242  0.414597   
         new_gen    0.883608  0.611091  0.305386   0.669197  0.400420   
top2     old_doc    1.216935  1.217802  1.154340   0.522058  0.513120   
         new_gen    1.207665  1.210525  1.152845   0.519280  0.511215   
share1   old_doc    1.142677  0.973876  0.779195   0.791412  0.614940   
         new_gen    1.150366  0.979569  0.777995   0.779056  0.606141   
share2   old_doc    1.052090  0.929623  0.734549   0.715778  0.574367   
         new_gen    1.059171  0.934044  0.738438   0.704763  0.567397   
skip1    old_doc    1.013716  0.885187  0.752828   0.696866  0.540950   
         new_gen    1.000150  0.869580  0.739407   0.676682  0.524919   
sparse2  old_doc    1.123184  0.947176  0.757206   0.777728  0.599609   
         new_gen    1.125088  0.954156  0.758486   0.766888  0.592985   
nolb     old_doc    1.091800  0.972075  0.785223   0.741787  0.599335   
         new_gen    1.098667  0.971448  0.787700   0.733385  0.588915   
overlb   old_doc    1.031012  0.891377  0.754451   0.771700  0.640649   
         new_gen    1.035592  0.894972  0.754558   0.762949  0.637130   

                            
cache_m                3.0  
model    mode               
baseline old_doc  0.461722  
         new_gen  0.452699  
32expert old_doc  0.501104  
         new_gen  0.489259  
top16    old_doc  0.199413  
         new_gen  0.190404  
top2     old_doc  0.461899  
         new_gen  0.459178  
share1   old_doc  0.460464  
         new_gen  0.451019  
share2   old_doc  0.421386  
         new_gen  0.411980  
skip1    old_doc  0.436299  
         

In [28]:
chr_dir = Path("../output/chr_mpq")

chr_df = pd.merge(
    pd.read_parquet(chr_dir / "m.parquet"), main_model_config, left_on="model", right_index=True
)

chr_df["model"] = chr_df["model"].astype(model_config.index.dtype)
chr_df

,model,is_decoder,dataset,method,cache_m,recall,ci_lb,ci_ub,model_name,model_abbr,model_type,model_num_params,model_num_layers,model_num_experts,model_top_k,model_attn
0,baseline,True,c4,LRU,0.015625,0.003556,0.003538,0.003574,baseline,BL,causal,1.43,8,64,8,sdpa
1,baseline,True,c4,LRU,0.031250,0.007462,0.007428,0.007494,baseline,BL,causal,1.43,8,64,8,sdpa
2,baseline,True,c4,LRU,0.046875,0.011688,0.011635,0.011736,baseline,BL,causal,1.43,8,64,8,sdpa
3,baseline,True,c4,LRU,0.062500,0.016184,0.016111,0.016251,baseline,BL,causal,1.43,8,64,8,sdpa
4,baseline,True,c4,LRU,0.078125,0.020975,0.020880,0.021062,baseline,BL,causal,1.43,8,64,8,sdpa
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
172387,overlb,True,science,Beladi,7.937500,0.999542,0.999535,0.999549,baseline,OB,causal,1.43,8,64,8,sdpa
172388,overlb,True,science,Beladi,7.953125,0.999684,0.999679,0.999689,baseline,OB,causal,1.43,8,64,8,sdpa
172389,overlb,True,science,Beladi,7.968750,0.999812,0.999809,0.999816,baseline,OB,causal,1.43,8,64,8,sdpa
172390,overlb,True,science,Beladi,7.984375,0.999922,0.999920,0.999924,baseline,OB,causal,1.43,8,64,8,sdpa


In [29]:
mcdf = (
    chr_df.query("method == 'LRU' and cache_m.isin([1.0, 2.0, 3.0])")
    .groupby(["model", "cache_m"], observed=True, as_index=False)[["recall"]]
    .mean()
)

mcdf.pivot(columns="cache_m", index="model", values="recall")

cache_m,1.0,2.0,3.0
model,,,
baseline,0.293701,0.474140,0.614081
32expert,0.255014,0.435932,0.579074
top16,0.439243,0.699631,0.892248
top2,0.099905,0.188884,0.253743
share1,0.270697,0.438061,0.566572
share2,0.239381,0.396527,0.514828
skip1,0.313387,0.498588,0.638797
sparse2,0.301056,0.479912,0.615446
nolb,0.436388,0.672001,0.823592


In [37]:
mmdf.merge(mcdf).query("not model.isin(['top16', 'top2'])").groupby(
    ["cache_m", "mode"], observed=True
)[["prefill_oh", "decode_oh", "prefill_ohr", "decode_ohr", "recall"]].corr()["recall"].unstack(
    -1
).drop(columns="recall").unstack(-1)

prefill_oh           decode_oh           prefill_ohr            \
mode       old_doc   new_gen   old_doc   new_gen     old_doc   new_gen   
cache_m                                                                  
1.0       0.094626  0.076122 -0.353779 -0.342702    0.188790  0.167119   
2.0       0.337051  0.217339 -0.419011 -0.440524    0.394074  0.298722   
3.0       0.131248  0.121281 -0.367251 -0.358451    0.243144  0.228427   

        decode_ohr            
mode       old_doc   new_gen  
cache_m                       
1.0      -0.196177 -0.188924  
2.0      -0.285584 -0.315164  
3.0      -0.298174 -0.289628